In [15]:
epochs, batch, lr = 100, 64, 1e-4
num_clients, in_channels, num_classes = 6, 3, 10
Dataset, Model, Prefix = 'CIFAR', 'VGG_3', 'TrainS'
Gdrive = '/content/drive/MyDrive/ColabNotebooks/SplitLearning3'
import torch; torch.cuda.get_device_properties(0)

_CudaDeviceProperties(name='Tesla T4', major=7, minor=5, total_memory=15101MB, multi_processor_count=40)

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import os; os.chdir(Gdrive)
%run library/Dataset.ipynb
%run library/Network.ipynb
%run library/Training.ipynb

In [5]:
dataset, testset = loadDataset()

100%|██████████| 170498071/170498071 [00:13<00:00, 13025072.77it/s]


Extracting /content/cifar-10-python.tar.gz to /content/
Files already downloaded and verified


In [14]:
class VGG_3(VGGNet):
  def __init__(self, in_channels, num_classes, dropout=(0.2,0.3)):
    super().__init__([(64,2)], 32, dropout[1])
    self.client = Conv2d(in_channels, 32, 3, 2, 1, dropout[0])
    self.server.append(nn.Flatten())
    self.server.append(nn.Linear(4096, 128))
    self.server.append(nn.BatchNorm1d(128))
    self.server.append(nn.Linear(128, num_classes))

#Imbalance

In [6]:
clients = np.array([1, 3, 9, 19, 30, 38])
clients = ratio_split(dataset, clients / clients.sum())

In [7]:
old_indices, new_indices = [0,2,3,5], [1,4]
old_clients = [clients[i] for i in old_indices]
new_clients = [clients[i] for i in new_indices]

In [35]:
network = train_splitnn(old_clients, (testset), 'sequence', 100, 'VGG_3')

  0%|          | 0/100 [00:00<?, ?it/s]

Output()

In [36]:
#train_sequence_all(network, new_clients, (testset), 100, 'VGG_3')
train_sequence_all(network, [], (testset), 100, 'VGG_3')

  0%|          | 0/100 [00:00<?, ?it/s]

Output()

In [32]:
def train_sequence_all(network, clients, testset, epochs=epochs, model=Model, batch=batch, lr=lr, caches=None):
  in_channels, num_classes, tester = loadTester(testset)
  models = [globals()[model](in_channels, num_classes) for c in clients]
  models = [c for c in network.clients] + models
  model = SplitNN([Client(m.client) for m in models], network.server)
  clients = [c.loader.dataset for c in network.clients] + clients
  model.initialize(clients, tester, epochs, batch, lr)

  for epoch in tqdm(range(epochs)):
    model.train_network(epoch, private=False, sequence=True, federate=False, caches=caches)
    model.evaluate()
    plot_progress(model.clients, locals())
  return model

def train_sequence_new(network, clients, testset, epochs=epochs, model=Model, batch=batch, lr=lr, caches=None):
  in_channels, num_classes, tester = loadTester(testset)
  models = [globals()[model](in_channels, num_classes) for c in clients]
  model = SplitNN_S([Client(m.client) for m in models], network)
  model.initialize(clients, tester, epochs, batch, lr)

  for epoch in tqdm(range(epochs)):
    model.train_network(epoch, caches=caches)
    model.evaluate()
    plot_progress(model.all_clients(), locals())
  return model

In [20]:
class SplitNN_S(SplitNN):
  def __init__(self, clients, exists):
    super().__init__(clients, exists.server)
    self.exists = exists

  def initialize(self, datasets, tester, epochs, batch, lr):
    super().initialize(datasets, tester, epochs, batch, lr)
    for c in self.exists.clients: c.accuracy = []
    self.cached = self.exists.cached

  def evaluate(self):
    super().evaluate()
    self.exists.evaluate()

  def all_clients(self):
    return self.exists.clients + self.clients

  def train_network(self, epoch, caches=None):
    self.server.train()
    model = self.exists.clients[-1].state_dict()
    for idx, client in enumerate(self.clients):
      client.load_state_dict(model)
      client.train()
      for images, labels in client.loader:
        client.zero_grad(), self.server.zero_grad()
        output = client(images)
        if caches: #Our
          indices = [torch.randperm(len(c)) for _,c in self.cached]
          images_cached = [c[0][i][:m] for c,i,m in zip(self.cached,indices,caches)]
          labels_cached = [c[1][i][:m] for c,i,m in zip(self.cached,indices,caches)]
          output = torch.cat([output] + images_cached)
          labels = torch.cat([labels] + labels_cached)
        output = self.server(output)
        loss = F.cross_entropy(output, labels)
        grads = self.server.backward(loss)
        client.backward(grads[:len(client.output)])
        client.optim_step(), self.server.optim_step()
      client.sched_step()
      model = client.state_dict()
    self.server.sched_step()

In [25]:
train_sequence_new(network, new_clients, (testset), 100, 'VGG_3')

  0%|          | 0/100 [00:00<?, ?it/s]

Output()

In [40]:
caches = [2 for i in range(len(old_clients))]
network = train_splitnn(old_clients, (testset), 'sequence', 100, 'VGG_3', caches=caches)

  0%|          | 0/100 [00:00<?, ?it/s]

Output()

In [41]:
caches = [2 for i in range(len(old_clients))]
train_sequence_new(network, new_clients, (testset), 100, 'VGG_3', caches=caches)

  0%|          | 0/100 [00:00<?, ?it/s]

Output()

In [ ]:
caches = [8 for i in range(len(old_clients))]
# private training with caching
network = train_splitnn(old_clients, (testset), 'private', caches=caches)
saveAccuracies(Gdrive, network.clients, 'ImB-Cac-1st')
saveAccuracies(Gdrive, train_private_new(network, new_clients, (testset), caches=caches).all_clients(), 'ImB-Cac-2nd')

  0%|          | 0/100 [00:00<?, ?it/s]

Output()

/content/drive/MyDrive/Colab Notebooks/SplitLearning3/data/CIFAR-VGG_11-Train2-6-ImB-Cac-1st.acc


  0%|          | 0/100 [00:00<?, ?it/s]

Output()

/content/drive/MyDrive/Colab Notebooks/SplitLearning3/data/CIFAR-VGG_11-Train2-6-ImB-Cac-2nd.acc
